1 - Set working enviroment

In [162]:
#import python packages
import geopandas as gpd
import pandas as pd
import os
from difflib import SequenceMatcher
import numpy as np

In [163]:
#import functions and config folders
from scripts.config import output_folder
from scripts.config import processed_folder
from scripts.functions import norm_gaul_levels

In [164]:
#Import from main_p.ipynb fao gaul 15 data
%store -r fao_gaul_15

In [165]:
#Import from main_p.ipynb fao gaul 24 data
%store -r fao_gaul_24

In [166]:
#set projected CRS for fao gaul 15 (same to fao gaul 24)
fao_gaul_15.crs = "EPSG:4326"

In [167]:
#set database index
fao_gaul_15.index = np.arange(fao_gaul_15.shape[0])
fao_gaul_24.index = np.arange(fao_gaul_24.shape[0])

2 - Delete from main databases N.A. values and store in excel files

In [168]:
#Extracting n.a. values 2015
na_15 = (fao_gaul_15['ADM2_NAME'] == 'Administrative unit not available') | \
     (fao_gaul_15['ADM0_NAME'] == 'Administrative unit not available') | \
     (fao_gaul_15['ADM1_NAME'] == 'Administrative unit not available')

na_15 = gpd.GeoDataFrame(fao_gaul_15[na_15])

na_15.to_excel(os.path.join(processed_folder,"na_2015.xlsx"),index=False)


In [169]:
#Extracting n.a. values 2024
na_24 = (fao_gaul_24['gaul2_name'] == 'Administrative Unit Not Available') | \
     (fao_gaul_24['gaul1_name'] == 'Administrative Unit Not Available') | \
     (fao_gaul_24['gaul0_name'] == 'Administrative Unit Not Available')

na_24 = gpd.GeoDataFrame(fao_gaul_24[na_24])

na_24.to_excel(os.path.join(processed_folder,"na_2024.xlsx"),index=False)

In [170]:
#clean 2015 data from n.a. values
to_drop_15 = na_15.index
fao_gaul_15_clean= gpd.GeoDataFrame(fao_gaul_15.drop(to_drop_15))

In [171]:
#clean 2024 data from n.a. values
to_drop_24 = na_24.index
fao_gaul_24_clean= gpd.GeoDataFrame(fao_gaul_24.drop(to_drop_24))

In [ ]:
#Check if the number of rows in fao_gaul_15_clean matches the expected count after removing n.a. values

print('count before cleaning:', len(fao_gaul_15))
print('count after cleaning:', len(fao_gaul_15_clean))
print('n.a. rows removed:', len(to_drop_15))

if len(fao_gaul_24_clean) + len(to_drop_24) != len(fao_gaul_24):
    print("Warning: The number of rows in fao_gaul_15_clean does not match the expected count after removing n.a. values.")
else:
    print("OK!! fao_gaul_15_clean n. of rows matches expected count after cleaning.")



count before cleaning: 38258
count after cleaning: 37108
n.a. rows removed: 1150
OK!! fao_gaul_15_clean n. of rows matches expected count after cleaning.


In [173]:
#Check if the number of rows in fao_gaul_15_clean matches the expected count after removing n.a. values

print('count before cleaning:', len(fao_gaul_24))
print('count after cleaning:', len(fao_gaul_24_clean))
print('n.a. rows removed:', len(to_drop_24))

if len(fao_gaul_15_clean) + len(to_drop_15) != len(fao_gaul_15):
    print("Warning: The number of rows in fao_gaul_15_clean does not match the expected count after removing n.a. values.")
else:
    print("OK!! fao_gaul_24_clean n. of rows matches expected count after cleaning.")

count before cleaning: 45523
count after cleaning: 45041
n.a. rows removed: 482
OK!! fao_gaul_24_clean n. of rows matches expected count after cleaning.


In [193]:
#store usefull variables
fao_gaul_15_count = len(fao_gaul_15)
fao_gaul_15_clean_count = len(fao_gaul_15_clean)

%store fao_gaul_15_count
%store fao_gaul_15_clean_count

fao_gaul_24_count = len(fao_gaul_24)
fao_gaul_24_clean_count = len(fao_gaul_24_clean)

%store fao_gaul_24_count
%store fao_gaul_24_clean_count

na_15_count = len(to_drop_15)
na_24_count = len(to_drop_24)

%store na_15_count
%store na_24_count

Stored 'fao_gaul_15_count' (int)
Stored 'fao_gaul_15_clean_count' (int)
Stored 'fao_gaul_24_count' (int)
Stored 'fao_gaul_24_clean_count' (int)
Stored 'na_15_count' (int)
Stored 'na_24_count' (int)


3- Perform spatial join on centroids (calculated on mercatore projection)

In [174]:
#Convert to mercatore and add centroids 2015
fao_gaul_15_clean['centroids'] = fao_gaul_15_clean.geometry.to_crs(epsg=3395).centroid

#set centroid as geometry
fao_gaul_15_clean = fao_gaul_15_clean.set_geometry("centroids")

In [175]:
#Convert to mercatore and add centroids 2024
fao_gaul_24_clean['centroids'] = fao_gaul_24.geometry.to_crs(epsg=3395).centroid

#set geometry to perform the join
fao_gaul_24_clean = fao_gaul_24_clean.set_geometry("centroids")

In [176]:
# spatial join on centroids (50000 m max distance))
spatial_join_50000m = fao_gaul_24_clean.sjoin_nearest(
    fao_gaul_15_clean,
    how="inner",
    max_distance=50000,
    distance_col="distance",
    lsuffix='24',
    rsuffix='15'
)

In [177]:
#create a dataframe based on geodataframe of spatial join
spatial_join = spatial_join_50000m[["gaul0_name", "gaul0_code",
                                    "gaul1_name", "gaul1_code",
                                    "gaul2_name", "gaul2_code",
                                    "ADM0_NAME", "ADM0_CODE", 
                                    "ADM1_NAME", "ADM1_CODE",
                                    "ADM2_NAME", "ADM2_CODE",
                                    "distance"]]

In [178]:
#show the first rows of the db
spatial_join.head()

#Clean duplicate and mantain only the closest match
spatial_join_clean = spatial_join.sort_values('distance', ascending=True).drop_duplicates(
    subset=["gaul2_name", "ADM2_NAME"],
    keep="first"
).reset_index(drop=True)

print('number of duplicates deleted:', (len(spatial_join))-(len(spatial_join_clean)))

number of duplicates deleted: 2780


In [179]:
# Calcola i codici unici matchati e unmatched per 2024 e 2015

# Codici unici matchati
#matched_24 = set(spatial_join_clean['gaul2_code'])
#matched_15 = set(spatial_join_clean['ADM2_CODE'])

# Tutti i codici presenti nei dati puliti
#all_codes_24 = set(fao_gaul_24_clean['gaul2_code'])
#all_codes_15 = set(fao_gaul_15_clean['ADM2_CODE'])

# Codici unmatched
#unmatched_24 = all_codes_24 - matched_24
#unmatched_15 = all_codes_15 - matched_15

# DataFrame delle geometrie unmatched
#un_s_match_24 = fao_gaul_24_clean[fao_gaul_24_clean['gaul2_code'].isin(unmatched_24)]
#un_s_match_15 = fao_gaul_15_clean[fao_gaul_15_clean['ADM2_CODE'].isin(unmatched_15)]

#print('number of spatial matched for 2024:', len(matched_24))
#print('number of spatial matched for 2015:', len(matched_15))
#print('number of spatial unmatched for 2024:', len(unmatched_24))
#print('number of spatial unmatched for 2015:', len(unmatched_15))
#print("Totale 2024:", len(all_codes_24))
#print("Totale 2015:", len(all_codes_15))
#print("Somma 2024:", len(matched_24) + len(unmatched_24))
#print("Somma 2015:", len(matched_15) + len(unmatched_15))


4 - String Match

In [180]:
#Normalize gaul levels
norm_spatial_join = norm_gaul_levels(spatial_join_clean)
print(len(norm_spatial_join))

38246


In [181]:
#eliminate prefix to improve string matching
norm_spatial_join['join_col'] = norm_spatial_join['join_col'].str.replace('municipality', '')
norm_spatial_join['join_col'] = norm_spatial_join['join_col'].str.replace('cityof', '')
norm_spatial_join['join_col'] = norm_spatial_join['join_col'].str.replace('city', '')
norm_spatial_join['join_col'] = norm_spatial_join['join_col'].str.replace('district', '')
norm_spatial_join['JOIN_COL'] = norm_spatial_join['JOIN_COL'].str.replace('municipality', '')
norm_spatial_join['JOIN_COL'] = norm_spatial_join['JOIN_COL'].str.replace('cityof', '')
norm_spatial_join['JOIN_COL'] = norm_spatial_join['JOIN_COL'].str.replace('city', '')
norm_spatial_join['JOIN_COL'] = norm_spatial_join['JOIN_COL'].str.replace('district', '')

In [182]:
# Define match ratio of names
def match_ratio(column1, column2, df):
    return df.apply(lambda row: SequenceMatcher(None, row[column1], row[column2]).ratio(), axis=1)

norm_spatial_join["match_ratio"] = match_ratio("join_col", "JOIN_COL", norm_spatial_join)

In [183]:
#rename columns for clarity
norm_spatial_join.rename(columns={"join_col": "join_col2024",
                                  "JOIN_COL" : "join_col2015",
                                  "gaul_code" : "gaul_code2024",
                                  "ADM_CODE" : "gaul_code2015",
                                  "gaul_name" : "gaul_name2024",
                                  "ADM_NAME" : "gaul_name2015"}, inplace=True)
norm_spatial_join.head()

,gaul_code2024,gaul_name2024,gaul_code2015,gaul_name2015,livello,distance,join_col2024,join_col2015,match_ratio
0,101,Algeria,4,Algeria,0,NaN,algeria,algeria,1.000000
3,237,Democratic People's Republic of Korea,67,Dem People's Rep of Korea,0,NaN,democraticpeoplesrepublicofkorea,dempeoplesrepofkorea,0.769231
12,139,Liberia,144,Liberia,0,NaN,liberia,liberia,1.000000
27,208,Puerto Rico,200,Puerto Rico,0,NaN,puertorico,puertorico,1.000000
29,247,Jordan,130,Jordan,0,NaN,jordan,jordan,1.000000


In [184]:
# Filter the DataFrame based on the conditions
# Keep records with match_ratio > 0.4, or match_ratio < 0.39 and distance < 2000, or where distance is NaN (gaul 0 and 1)

filter_par = (
    (norm_spatial_join['match_ratio'] > 0.6) |
    ((norm_spatial_join['match_ratio'].between(0.4, 0.59) & (norm_spatial_join['distance'] < 2000))) |
    (norm_spatial_join['distance'].isna())
)


# Filter the DataFrame based on the conditions
norm_spatial_join_filtered = norm_spatial_join[filter_par]
out = norm_spatial_join[~filter_par]

# sort and keep record with the higher match_ratio
norm_spatial_join_filtered = norm_spatial_join_filtered.sort_values('match_ratio', ascending=False)
norm_spatial_join_filtered = norm_spatial_join_filtered.drop_duplicates(subset='gaul_code2024', keep='first')

# Same for gaul 2015
norm_spatial_join_filtered = norm_spatial_join_filtered.sort_values('match_ratio', ascending=False)
norm_spatial_join_filtered = norm_spatial_join_filtered.drop_duplicates(subset='gaul_code2015', keep='first')

print(len(norm_spatial_join_filtered))

# Rimuovi i duplicati per entrambe le colonne
out_nodup = out.drop_duplicates(subset=['gaul_code2015', 'gaul_code2024'], keep='first')

print(len(out_nodup))


30059
7122


In [185]:
#save the dataframe to excel
norm_spatial_join.to_excel(os.path.join(processed_folder,"spatial_and_string_match.xlsx"),index=False)
norm_spatial_join_filtered.to_excel(os.path.join(processed_folder,"spatial_and_string_match_filtered.xlsx"),index=False)


In [194]:
gaul2_matched = norm_spatial_join_filtered[norm_spatial_join_filtered['livello'] == 2]
print(len(gaul2_matched))

27819


In [196]:
#store usefull variables

matched_count = len(gaul2_matched)

%store matched_count

Stored 'matched_count' (int)


In [187]:
#check un matched rows
mask = (
    ~(fao_gaul_15_clean['ADM2_CODE']).isin(norm_spatial_join_filtered['gaul_code2015'])
)
fao_gaul_15_unmatched = fao_gaul_15_clean[mask]

len(fao_gaul_15_unmatched)

9285

In [188]:
fao_gaul_15_unmatched.to_excel(os.path.join(processed_folder,"unmatched_2015.xlsx"),index=False)

In [ ]:
#check un matched rows
mask_24 = (
    ~fao_gaul_24_clean['gaul2_code'].isin(norm_spatial_join_filtered['gaul_code2024']) 
)
fao_gaul_24_unmatched = fao_gaul_24_clean[mask_24]
len(fao_gaul_24_unmatched)

17222

In [190]:
fao_gaul_24_unmatched.to_excel(os.path.join(processed_folder,"unmatched_2024.xlsx"),index=False)

In [197]:
#store usefull variables
unmatched_count_15 = len(fao_gaul_15_unmatched)
unmatched_count_24 = len(fao_gaul_24_unmatched)

%store unmatched_count_15
%store unmatched_count_24   

Stored 'unmatched_count_15' (int)
Stored 'unmatched_count_24' (int)


In [191]:
https://sankeymatic.com/build/?i=PTAEFEDsBcFMCdQDEA2B7A7gZ1AI1tBrLJKAHJoAmsWANKCgJYDWso0AFo1gFwBQIUEOHCAymgCu8AMZsA2gEEAsgHkAqmQAqAXVCaAhvADmBPnyT60oAOL6JKUACZQACkcAGAIwBWAJSg5AGYAdk93AA5dAGEUWH1SH3NLGzsHZzcvPwDPH3ddMgA6BQLEvhi4hO8Ax2DwzwBOXVEAB31oRn0HAFs26Q5YSlBE8vihqrl6x3DvXTVIHug%2BsbMkq1t7J1cPRwAWfzkd73cdz2jY0d3VlI307b2AnfDHfKKCpx2zEdJd6tqGpta7U6oAWfQG7zK52%2BO2ywUc8Nm816HAhAjAAE1JKBpKMsARQPpyFRYAByHDSNDoeD0JisdhcXiQirvUAAYnc3nCK0Eajx9LYFJg8EpOHw6Aw7Cs0gkWGgaC6jAAXrA0aAAJ6SRCUDpGeD6LpkgnNZpxPWQWQFS18LBKtgSgBs7ncfCEKMdzp6xkYpAcnkcLtAiD9AegQy5QlwTmduCM2MpaEQrIAZinUyGzVhWvASKGyHxIMTQBLg67QEcA5n9NJvbHgt4A7gE9REM6hJxYF02PoAxSqWzwgPBwG0K1q9A1UM%2BEnxdipAA3NpSNjuAr1oTe%2DrwRjQJPCrqgSTQG3UAC03p78cT9WvN%2BHo%2B3E5Xhz4KH0GokoabCAJH%2DlbUY0gBgAVjK7RJhqW5GN6OB5kIIGyow4EkJQMEBtmc4IHiur6M0KKwQS0DQFWXDmvKzSxHAspWJApo0NAL76GK5KXmyTpsa2oBcNQ5ABlwRgcEw%2DGhiudYBkmaAwEmVZsFg8RYCeeJbkmAZMDRlbVpAsYrv6QjZq%2B7QYTaypDGEAY9EYkCIROYTOq%2BYqQPqXbGqaODohWtpDPaAZEIwQmgDsToMWKC4oBITkmoYrlifYKDNNm1Y2hJoBuUIzRoDa7RJWKmDebAvkcKGAW2YxsCxel26MEldhyp0vmkBxWBgp2P5ymJjDwLKeCwOJ2YNtm%2BjMGl3qFXwIVhaAPULKAJK0AUJIBnF3WMAAHtN81CFgEgpita18O2nZoCmeKhoSXkRqAHHSBdAaDB6BCEp2MCVZAsmQKwaoLAByUqdw7RdLgnTxLIrlAA

SyntaxError: invalid decimal literal (843037397.py, line 1)